In [1]:
#
#
#
#
from scicrapy import *
from urllib   import request
from time     import sleep
from bs4      import BeautifulSoup

import numpy  as np
import pandas as pd
import requests
import re

# Links for IOP search

IOP_S  = "https://iopscience.iop.org/nsearch?terms="
IOP_E  = "&nextPage=2&previousPage=-1&currentPage="
IOP_P  = "&searchDatePeriod=anytime&orderBy=relevance&pageLength=50"

# Links for NATURE search

NANO_S = 'https://nano.nature.com/search?term=freeText%3A'
NANO_E = '&sort-by=relevance&page-number='
NANO_P = '&workflow=article'


"""==============================================================================================================================="""

# -format- returns a string that can be readed as a query to IOP or NANO

def format(query):
    pre_form = query.split(" ")
    return "+".join(pre_form)


"""==============================================================================================================================="""

# Some pages may have problems if requested with -urllib- or -requests-, SwitchKitchen is implemented to
# switch the way we obtain a BeautifulSoup object


def SwitchKitchen(uri, kitchen = "requests", parse = "lxml"):

    # Request with --requests.get(url)--

    if   kitchen == "requests":
         response = requests.get(uri)
         soup     = BeautifulSoup(response.text, parse)

    elif kitchen == "urllib":

    # Request with --request.urlopen(url)--

         response = request.urlopen(uri)
         soup     = BeautifulSoup(response, parse)
    else:
        raise Exception("Problema en la elección del método mara obtener la respuesta del servidor.")
    return soup;

"""==============================================================================================================================="""

# /Souparticles : Returns a list of BeautifulSoup objects that contain articles

# /material     : Is a str. containing the search term
# /page         : The page in which we are searching
# /max_try      : The maximum tries before givin up, commonly this is a result of a 400 status response from the server
# /slpy         : Is the time between requests, recommended to prevent overloading requested server


def souparticlesIOP(query, page, max_int = 5, slpy = 3):
    # At the beginning our IOP Soup is a None object
    soupIOP = None
    ints  = 0

    while ints<max_int:    
    # We try to catch some content from the server

        try:
            uri      = IOP_S+ format(query) + IOP_E + str(page) + IOP_P
            # If we catch a 200 response we can make a Beautifoul Soup (BS), parsing with lxml
            soupIOP  = SwitchKitchen(uri,kitchen = "urllib")
            break
        except:
            # If we catch an error, we can retry
            print("Reintentando pag: " + str(page))
            sleep(slpy)
            pass
        finally:
            ints +=1
    
    # For soupIOP == None this process needs to be repeated
    if(soupIOP==None):
        print( "Por favor reinicie la busqueda para pag: " + str(page))
        return None;
    else:
        # Returning the articles in a BS object
        print("Añadida pag: " + str(page))
        return soupIOP.find_all('div', {"class" : "art-list-item-body"})

"""==============================================================================================================================="""

def souparticlesNANO(query, page, max_int = 5, slpy = 3):
    nanosoup = None
    ints=0
    while ints<max_int:
        try:
            uri      = NANO_S+'"'+format(query)+'"'+NANO_E+str(page)+NANO_P
            nanosoup = SwitchKitchen(uri, kitchen = "requests")
            break
        except:
            print("Reintentando pag: " + str(page))
            sleep(slpy)
            pass
        finally:
            ints +=1
    if(nanosoup==None):
        print( "Por favor reinicie la busqueda para pag: " + str(page))
        return None;
    else:
        print("Añadida pag: " + str(page))
        return nanosoup.find_all('li', {"class" : "Results_listItem" })

"""==============================================================================================================================="""

# -Pages- returns a list of pages, each element contains the articles within that page

def PagesIOP(query):
    raw_pages   = []
    init_uri  = IOP_S+ format(query) + IOP_E + str(1) + IOP_P
    init_soup = SwitchKitchen(init_uri,kitchen = "urllib")
    end_pages = init_soup.findChild("p",{"class":"pgs small"}).get_text().split(" ")[2]
    for i in range(1, int(end_pages)+1):
        raw_pages.append(souparticlesIOP(query, i))
    return raw_pages;

"""==============================================================================================================================="""

def PagesNANO(query, cut = 12):
    raw_pages = []
    init_uri  = NANO_S+'"'+format(query)+'"'+NANO_E+str(1)+NANO_P
    init_soup = SwitchKitchen(init_uri,kitchen = "requests")
    end_pages = init_soup.findChild("span",{"class":"Pagination_numOfPages"})
    num_of_pgs= int(end_pages.get_text().replace("\n","").replace(" ","").replace(",",""))
    if num_of_pgs<12:
        for i in range(1,num_of_pgs+1):
            raw_pages.append(souparticlesNANO(query, i))
    else:
        for i in range(1, cut + 1 ):
            raw_pages.append(souparticlesNANO(query, i))
    return raw_pages;

"""==============================================================================================================================="""

# -stringify- returns a list of articles, each article containing it´s features, that can be readed as normal strings

def stringifyIOP(raw_pages):
    arts_db = []
    for page in raw_pages:
        for element in page:
            try:
                title     =     element.findChild('h2' , {"class"   : "art-list-item-title"}).findChild("a")          # 0 title
                abstract  =     element.findChild('div', {"class"   : "article-text view-text-small"}).findChild("p") # 1 abstract
                DOI       =     element.findChild('a'  , {"class"   : "mr-2"})                                        # 2 DOI
                journal   =     element.findChild('em')                                                               # 3 Journal
                #vol       =     element.findChild('b')                                                              
                authors   =     element.find_all('span', {"itemprop": "author"})                                      # 4 auths

                yearfind  =     element.find_all("p"   , { "class" :"small art-list-item-meta"})                      # 5 year
                year      =     int(yearfind[1].text[14:19])

                arts_db.append([title.text ,abstract.text, DOI.text, journal.text,[auth.getText() for auth in authors], year])
            except:
                pass

    return arts_db;


"""==============================================================================================================================="""

def stringifyNANO(raw_pages):
    arts_db = []
    for page in raw_pages:
        for element in page:
            try:
                title       =     element.findChild("h2").getText().replace("\n","")                                # 0 Title
                miscel      =     element.find_all("p")

                abstract    =     miscel[1].getText()                                                               # 1 Abstract
                DOI         =     element.findChild('div', {"class": "Doi"}).getText()                              # 2 DOI
                journal     =     miscel[0].findChild("strong").getText()                                           # 3 Journal

                #cited       =     miscel[2].getText()

                authstag    =     element.findChild("ul", {"class":"PipeSepList Author"})                           # 4 Auths
                authors_tag =     authstag.find_all("li")
                authors_str = [author.getText().replace("\n","") for author in authors_tag]

                begi        =     miscel[0].getText().find("(")
                year        =     miscel[0].getText()[begi+1:begi+5]                                                # 5 Year

                arts_db.append([title, abstract, DOI, journal, authors_str, year])
            except:
                pass

    return arts_db;

"""==============================================================================================================================="""

# -query- returns a list of articles related to the search-term term*, note that this returns both IOP and NANO articles

def query(terms):
    all_arts = []

    query_raw_iop  = PagesIOP(terms)
    query_raw_nano = PagesNANO(terms)

    IOP_DB         = stringifyIOP(query_raw_iop)
    NANO_DB        = stringifyNANO(query_raw_nano)

    for article in NANO_DB:
        all_arts.append(article)
    for article in IOP_DB:
        all_arts.append(article)

    return all_arts;

"""==============================================================================================================================="""

def search_kwords(arts, kwords):
    candidates = []
    count = 0
    for art in arts:
        if kwords in art[1]:
            candidates.append(art)
            count +=1
    print("Coincidences: " +str(count))
    return candidates;


"""==============================================================================================================================="""


def pandas_dataframe(all_arts, name):
    test_arts = {}
    for i in range(0,6):
        ls = []
        for j in range(0,len(all_arts)):
            ls.append(all_arts[j][i])
        if i == 0:
            test_arts.update(Title        = ls)
        elif i == 1:
            test_arts.update(Abstract     = ls)
        elif i == 2:
            test_arts.update(DOI          = ls)
        elif i == 3:
            test_arts.update(Journal      = ls)
        elif i == 4:
            test_arts.update(Auths        = ls)
        else:
            test_arts.update(Year         = ls)
    pd.DataFrame(test_arts).to_pickle("mispepinillos/{}.pkl".format(name))
    return pd.DataFrame(test_arts);

# To read a pandas dataframe local_datarame= pandas.read_pickle("folder/file.pkl")

"""==============================================================================================================================="""

def geve_art(art_num, dataframe):
    print(dataframe.iloc[art_num]["Abstract"].replace("\n", ""))
    print("\n")
    print(dataframe.iloc[art_num]["Title"].replace("\n", ""))
    print("\n")
    print(dataframe.iloc[art_num]["Year"])
    print("\n")
    print(dataframe.iloc[art_num]["DOI"])
    print("\n")
    print(dataframe.iloc[art_num]["Journal"])
    return;

"""==============================================================================================================================="""

def dbsearch(word, db):
    ls = []
    for i in range(0,len(db)):
        if word in local_znte.iloc[i]["Abstract"].lower():
            ls.append(i)
        else:
            pass
    new_db = db.iloc [ls]
    new_index = len(new_db)
    new_db.set_index(np.arange(0,new_index),inplace=True)
    return new_db; 

"""==============================================================================================================================="""

def cite_manager(art, com, db = pd.Series([])):
    if db.empty:
        art = art.to_frame().transpose()
        art = art.assign(comment = [com])
        return art;
    else:
        art = art.to_frame().transpose()
        art = art.assign(comment = [com])
        db  = pd.concat([db,art], ignore_index=True, copy=False)
        return db;


In [78]:
# year_sorted = mex_db.sort_values(by="Year",ascending=False)
# year_znte = local_znte.sort_values(by="Year",ascending=False)

local_znte= pd.read_pickle("mispepinillos/znte.pkl")

sem      = dbsearch("sem", local_znte)
sem_abl  = dbsearch("sputt", sem)
ablation = dbsearch("ablation", local_znte)
sputter  = dbsearch("sputt", local_znte)
cu_sp    = dbsearch("sputt", cu)


In [65]:
ablation_cites = cite_manager(sputter.iloc[4], "Aplicaciones con ZnO para electronica, como la ref Nitrogen --doping of ZnTe--", ablation_cites)

In [79]:
ablation_cites

,Title,Abstract,DOI,Journal,Auths,Year,comment
0,Ovonic threshold switching in polycrystalline...,Chalcogenide materials of the amorphous phase ...,https://doi.org/10.1088/1361-6528/aafe13,Nanotechnology,"[Taeho Kim, Youngjae Kim, Ingwan Lee, Dayoon L...",2019,ZnTe por RF 2019 temperaturas de 400°
1,Effect of in situ annealing on structure and ...,"An improvement in morphology, crystallinity, a...",https://doi.org/10.1088/1742-6596/59/1/152,J. Phys.: Conf. Ser.,"[Takaaki Orii, Makoto Hirasawa, Takafumi Seto]",2007,Mejoramiento de la calidad cristalina ZnTe pld...
2,Properties of Nitrogen-Doped Zinc Telluride Fi...,Zinc telluride (ZnTe) films have been deposite...,10.1007/s11664-017-5502-0,Journal of Electronic Materials,"[Tushar M. Shimpi, Jennifer Drayton, Drew E. S...",2017,Disminución de la calidad cristalina con la pr...
3,Nitrogen doping of ZnTe for the preparation of...,Nitrogen-doped p-type zinc telluride (p-ZnTe) ...,10.1007/s10853-013-7438-y,Journal of Materials Science,"[A. E. Rakhshani, S. Thomas]",2013,Disminución del gap (1.4 eV) N= 5/100 con un a...
4,Photovoltaic-pyroelectric effect coupled broad...,Broadband photodetection is widely used in com...,10.1016/j.nanoen.2019.05.050,Nano Energy,"[Daotong You, Chunxiang Xu, Wei Zhang, Jie Zha...",2019,"Aplicaciones con ZnO para electronica, como la..."


In [92]:
ablation_cites.to_pickle("mispepinillos/cite_manager_znte.pkl")

In [91]:
geve_art(0,cu_sp)

BaCuSeF/ZnTe is a model system to investigate physical and chemical properties of the interfaces of non-oxide wide-bandgap p-type semiconductors with materials used in chalcogenide solar cells. The BaCuSeF/ZnTe interface was studied using electron microscopy and photoelectron spectroscopy. Both techniques indicate that Se and Cu from BaCuSeF diffuse into ZnTe creating an interdiffused layer between these two materials. The interdiffusion may be attributed to the differences in materials formation enthalpies and to Fermi level pinning in BaCuSeF.


Interdiffusion at the BaCuSeF/ZnTe interface


2011


10.1016/j.tsf.2010.12.125


Thin Solid Films


In [45]:
geve_art(5,sputter)

Chalcogenide materials of the amorphous phase with low band gaps were reported to show Ovonic threshold switching (OTS), making them suitable for selection devices in cross-point memory arrays. Herein, we report that ZnTe films with polycrystalline structures show OTS behavior. Nearly stoichiometric ZnTe thin films were deposited by an RF sputtering method. X-ray diffraction analysis indicated that the films were polycrystalline. The optical band gaps of the ZnTe films were estimated as 2.2 eV from UV–visible spectroscopy transmittance measurements. Photoluminescence measurements indicated the existence of deep-level defects in the ZnTe thin films. Although these ZnTe films have a polycrystalline structure with a relatively high band gap, I–V profiles show OTS characteristics, with a selectivity of over 104, fast threshold switching time in the sub-10 ns scale, and thermal stability up to 400 °C. ZnTe also shows switching endurance for more than 109 cycles without Vth drift, maintainin

In [78]:
geve_art(59,year_znte)

In this work, ZnTe and ZnTe:Cu films were obtained by pulsed laser deposition using the co-deposition method. ZnTe and Cu2Te were used as targets and the shots ratio were varied to obtain 0.61, 1.47, 1.72, and 3.46% Cu concentration. Doping of ZnTe films with Cu was performed with the purpose of increasing the p-type carrier concentration and establishing the effect of concentration of Cu on structural, optical, and electrical properties of ZnTe thin films to consider their potential application in electronic devices. According to X-ray diffraction, X-ray photoelectron spectroscopy, UV–visible spectroscopy, and Hall effect results, ZnTe and ZnTe:Cu films correspond to polycrystalline zinc–blende phase with preferential orientation in (111) plane. Optical characterization results indicate that as-deposited films (band gap = 2.16 eV) exhibit a band gap decrease as function of the increase of Cu concentration (2.09–1.64 eV), while, annealed films exhibit a decrease from 1.75 to 1.46 eV, a